In [2]:
import os
import pandas as pd
from datetime import date, timedelta
import json
import requests
from api_keys import g_key
import gmaps
import numpy as np

gmaps.configure(api_key = g_key)

# Adding options to display all of the rows of DataFrame w/o truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Range of latitudes and longitudes
range_of_latitude = (-90, 90)
range_of_longitude = (-180, 180)

output_path = os.path.join('..','Outputs')
images_path = os.path.join('..','Outputs', 'Images')

# The weather data was acquired on 02/01/20
# To assure accuracy, Change this number to match the exact date of data retrieval
days_since_data_request = 4

weather_df = pd.read_csv(output_path + '/weather_df.csv')
weather_df.head()

,City,Country,Country Code,Date,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,Vaini,Tonga,TO,1580600753,-21.20,-175.20,84.20,70.0,40.0,8.05
1,Grindavik,Iceland,IS,1580600944,63.84,-22.43,28.40,86.0,75.0,10.29
2,Bermejo,"Bolivia, Plurinational State of",AR,1580600945,-32.88,-68.80,95.00,21.0,40.0,4.70
3,Tocopilla,Chile,CL,1580600945,-22.09,-70.20,60.04,57.0,3.0,3.00
4,Arraial Do Cabo,Brazil,BR,1580600865,-22.97,-42.03,80.11,85.0,88.0,10.36


In [3]:
coordinates = weather_df[['Latitude','Longitude']]
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0 auto 0 auto'
}
print(f'Maximum Humidity: {weather_df["Humidity"].max()}')
fig = gmaps.figure(zoom_level = 2, center = (40,10), layout=figure_layout, display_toolbar = False)
humidity_layer = gmaps.heatmap_layer(coordinates, \
                                     weights= weather_df['Humidity'], \
                                     dissipating=False, \
                                     max_intensity=weather_df['Humidity'].max(), point_radius=3)
fig.add_layer(humidity_layer)
fig

Maximum Humidity: 100.0


Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='0px', wi…

## My Ideal Vacation Conditions:-
1. **Not too humid**: Humidity < 50%
2. **Gentle to Moderate Breeze**: Not too high winds, Up to Moderate winds is good: Wind speed should be less than 16 mph.
3. **Some clouds are nice. Make good pictures**: Cloudiness should be less than 40%.
4. **Not too cold, and not too hot**: Max Temp should be between 50 F to 70F

In [7]:
weather_df = pd.read_csv(output_path + '/weather_df.csv')
weather_df.head()
# print(len(weather_df))

# 1. Humidity Condition
weather_df.drop(weather_df[weather_df['Humidity'] >= 50].index, inplace = True)
# print(f'Length of df after incorporating Humidity: {len(weather_df)}')

# # Sanity check for Humidity
# print(weather_df['Humidity'].max())
# print(weather_df['Humidity'].min())

# 2. Wind Speed
weather_df.drop(weather_df[(weather_df['Wind Speed'] >= 16)].index, inplace=True)
# print(f'Length of df after incorporating Wind Speed: {len(weather_df)}')

# # Sanity Check for Max Temp
# print(weather_df['Wind Speed'].max())
# print(weather_df['Wind Speed'].min())

# 3. Cloudiness Condition
weather_df.drop(weather_df[(weather_df['Cloudiness'] > 40)].index, inplace=True)
# print(f'Length of df after incorporating Cloudiness: {len(weather_df)}')

# # Sanity Check for Max Temp
# print(weather_df['Cloudiness'].max())
# print(weather_df['Cloudiness'].min())

# 4. Temperature Condition
weather_df.drop(weather_df[(weather_df['Max Temp'] < 50) | (weather_df['Max Temp'] > 70)].index, inplace=True)
# print(f'Length of df after incorporating Max Temp: {len(weather_df)}')

# # Sanity Check for Max Temp
# print(weather_df['Max Temp'].max())
# print(weather_df['Max Temp'].min())

# 5. Drop any rows with null values
weather_df.dropna(how = 'any', inplace = True)

print(f'There are {len(weather_df)} locations matching your ideal vacation conditions.')
weather_df

There are 21 locations matching your ideal vacation conditions.


,City,Country,Country Code,Date,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
41,Mosquera,Colombia,CO,1580600950,4.71,-74.23,64.40,48.0,40.0,4.70
63,Busselton,Australia,AU,1580601006,-33.65,115.33,70.00,41.0,0.0,12.57
82,Estelle,United States,US,1580601008,29.85,-90.11,59.00,47.0,1.0,6.93
91,Gatesville,United States,US,1580601010,31.44,-97.74,69.80,19.0,1.0,4.70
116,Eunice,United States,US,1580600999,30.49,-92.42,60.80,35.0,1.0,5.82
135,Saint George,Bermuda,US,1580601069,37.10,-113.58,64.40,20.0,1.0,3.85
185,Kandi,Benin,BJ,1580601128,11.13,2.94,68.23,14.0,0.0,5.32
188,Ponta Do Sol,Cabo Verde,PT,1580601129,32.67,-17.10,70.00,36.0,20.0,8.05
191,Miles City,United States,US,1580601129,46.41,-105.84,62.01,32.0,1.0,14.99
194,Abu Zabad,Sudan,SD,1580601130,12.35,29.25,59.38,20.0,0.0,13.62


In [ ]:
target_name = 'hotel'

request_counter = 0
record_number = 0

# build info box list for the map
info_box_list = []

weather_df_zipped = zip(weather_df.City, weather_df.Country, weather_df.Latitude, weather_df.Longitude)

hotel_df = pd.DataFrame({'City':[],\
                           'Country':[],\
                           'Name':[],\
                           'Rating':[],\
                           'Latitude':[],\
                           'Longitude':[]})

for city, country, city_latitude, city_longitude in weather_df_zipped: 
    target_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={target_name}'
    input_url = f'&inputtype=textquery&fields=name,rating,geometry&locationbias=circle:5000@{city_latitude},{city_longitude}&key={g_key}'
    
    request_url = target_url + input_url
    response = requests.get(request_url)
    
    request_counter = request_counter + 1
    record_number = record_number + 1
    
    if response:
        print(f'Record# {record_number}/{len(weather_df)} : {city.title()}, {country} : PROCESSED')
        response = response.json()
#         print(json.dumps(response, indent = 4, sort_keys = True)) 
        
        hotel_name = response['candidates'][0]['name']
        try:
            hotel_name = response['candidates'][0]['name']
            hotel_rating = response['candidates'][0]['rating']
            hotel_lat = response['candidates'][0]['geometry']['location']['lat']
            hotel_lng = response['candidates'][0]['geometry']['location']['lng']
            response_items_df = pd.DataFrame([[city,country,hotel_name,hotel_rating,hotel_lat,hotel_lng]], \
                                              columns=list(hotel_df.columns))
            hotel_df = hotel_df.append(response_items_df)

        except:
            continue
    else:
        print(f'Record# {record_number}/{len(weather_df)} : {city.title()}, {country} : NOT FOUND')
        continue
    
#     break
    

In [66]:
hotel_df

,City,Country,Name,Rating,Latitude,Longitude
0,Mosquera,Colombia,Habitel Hotel,4.5,4.688235,-74.130540
0,Busselton,Australia,The Sebel Busselton,4.4,-33.659699,115.278824
0,Estelle,United States,The Queen Anne Hotel,3.9,29.937247,-90.075340
0,Gatesville,United States,Holiday Inn Express & Suites Gatesville - N. F...,4.3,31.422527,-97.716021
0,Eunice,United States,Holiday Inn Express Eunice,3.6,30.494396,-92.442698
0,Saint George,Bermuda,TownePlace Suites by Marriott St. George,4.4,37.103794,-113.551068
0,Kandi,Benin,HOTEL GB,4.5,11.130519,2.932578
0,Ponta Do Sol,Cabo Verde,Estalagem Da Ponta Do Sol,4.7,32.679639,-17.103093
0,Miles City,United States,Best Western War Bonnet Inn,4.0,46.398736,-105.823180
0,Abu Zabad,Sudan,Holiday Inn Minneapolis Airport SE - Eagan,3.9,44.858416,-93.166451


In [72]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [73]:
# Only plot the countries with ideal vacation conditions
coordinates = hotel_df[['Latitude','Longitude']]
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0 auto 0 auto'
}
print(f'Maximum Humidity: {weather_df["Humidity"].max()}')
fig = gmaps.figure(zoom_level = 2, center = (40,10), layout=figure_layout, display_toolbar = False)
humidity_layer = gmaps.heatmap_layer(coordinates, \
                                     weights= weather_df['Humidity'], \
                                     dissipating=False, \
                                     max_intensity=weather_df['Humidity'].max(), point_radius=5)
fig.add_layer(humidity_layer)
fig

marker_layer = gmaps.marker_layer(coordinates, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Maximum Humidity: 48.0


Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='0px', wi…